In [ ]:
import numpy as np
from scipy.integrate import quad
import pandas as pd
from scipy.optimize import least_squares
from scipy.optimize import root, brentq, minimize

import plotly.graph_objs as go
from plotly.offline import iplot

import cufflinks as cf
cf.go_offline()

import plotly.io as pio


from scipy.stats import norm
###
import time
_tstart_stack = []

def tic():
    _tstart_stack.append(time.time())

def toc(fmt="Elapsed: %s s"):
    print(fmt % (time.time() - _tstart_stack.pop()))
###

In [ ]:
def N(S):
    return norm.cdf(S)

## Define the functions outside the scope

def dplus(S, K, sigmaT):
    return np.log(S/K) / sigmaT + sigmaT * 0.5 

def dminus(S, K, sigmaT):
    return np.log(S/K) / sigmaT - sigmaT * 0.5

# Price functional for a vanilla option
def Price_func(S, K, T, Rd, Rf, sigmaT, Type = 1):
    result = 0
    if sigmaT == 0:
        result = np.maximum( Type * (S * np.exp(-Rf * T) - K * np.exp(-Rd * T)),0)
    else:
        term1 = np.exp(-Rf * T) * S * N(Type * dplus(np.exp(-Rf * T) * S, np.exp(-Rd * T) * K, sigmaT))
        term2 = np.exp(-Rd * T) * K * N(Type * dminus(np.exp(-Rf * T) * S, np.exp(-Rd * T) * K, sigmaT))
        result = Type * (term1 - term2)
    return result
Price = np.vectorize(Price_func)

# implied volatility
def Impliedvol_func(S, K, T, Rd, Rf, price, Type = 1):
    # Choose which root to take into account whether you are deep in the money or not.
    def Root(x):
        return Price(S, K, T, Rd, Rf, x * T ** 0.5, Type) - price
    result = 0
    try:
        result = brentq(Root, 0.00001, 30)
    except Exception:
        print('There is a problem here with brentq')
        result = root(Root, 0.2).x[0]
    return result
Impliedvol = np.vectorize(Impliedvol_func)

Pareto Glue Pricing RS model 

In [ ]:
# my pricing formula 
def frechet_price_func(S, K, T, Rd, Rf, sigma0, sigma1, alpha, kappa, Type=1):
    def integrand(t):
        weight = alpha * t**(-1-alpha) * np.exp(-t**(-alpha))
        premium = (1 - np.exp(-t**(-alpha))) ** kappa
        bsprice = Price(S * (1 + kappa) * premium, K, T, Rd, Rf,(sigma0 ** 2 * t + sigma1 ** 2 * (T-t)) ** 0.5, Type)
        return weight * bsprice

    factor1 = Price(S * ((1 - np.exp(-T**(-alpha))) ** kappa), K, T, Rd, Rf, sigma0 * T ** 0.5, Type) * (1 - np.exp(-T**(-alpha)))
    factor2, err = quad(integrand, 0, T)
    return factor1 + factor2
frechet_price = np.vectorize(frechet_price_func)

s != 0

In [ ]:
# my pricing formula 
# fixed s=2 -- same freedome
def frechet_price_func2(S, K, T, Rd, Rf, sigma0, sigma1, alpha, kappa, s=2, Type=1):
    def integrand(t):
        weight = (alpha/s) * (t/s)**(-1-alpha) * np.exp(-(t/s)**(-alpha))
        premium = (1 - np.exp(-(t/s)**(-alpha))) ** kappa
        bsprice = Price(S * (1 + kappa) * premium, K, T, Rd, Rf,(sigma0 ** 2 * t + sigma1 ** 2 * (T-t)) ** 0.5, Type)
        return weight * bsprice

    factor1 = Price(S * ((1 - np.exp(-(T/s)**(-alpha))) ** kappa), K, T, Rd, Rf, sigma0 * T ** 0.5, Type) * (1 - np.exp(-(T/s)**(-alpha)))
    factor2, err = quad(integrand, 0, T)
    return factor1 + factor2
frechet_price2 = np.vectorize(frechet_price_func2)

In [ ]:
def Sigma_Model(x, S, K, T, Rd, Rf):
    """
    sigma0, sigma1, alpha, kappa, 
    
    x[0]: sigma0
    x[1]: sigma1
    x[2]: alpha
    x[3]: kappa
    """
#     price = frechet_price(S, K, T, Rd, Rf, x[0], x[1], x[2], x[3]) # s=0

    price = frechet_price2(S, K, T, Rd, Rf, x[0], x[1], x[2], x[3]) # s!=01
    return Impliedvol(S, K, T, Rd, Rf, price)

In [ ]:
# fit with sigma lower bar
def Fit_BF_sigmalow(OBJ, INPUT, GUESS, S, T, Rd, Rf, Verbose = 2):
    
    # By assumption OBJ = [SxP, SxP, SATM, SxC, SxC]
    # And INPUT = [KxP, KxP, KATM, KxC, KxC]

    # we first get the dimension
    n = OBJ.size

    BOUNDS = ([np.minimum(0.0001, OBJ[2]*0.3) , OBJ[2], 0.0001, -0.8], 
              [OBJ[2],                               2,  10,     0.8]) ## change! alpha upper bound
    
    # Check that the Guess is correct
    GUESS[1] = np.maximum(GUESS[1], OBJ[2]) # for sigma upper [1]
    GUESS[0] = np.maximum(np.minimum(GUESS[0], OBJ[2]*0.5), 0.0001) # for sigma lower

    ###
    Sigma = Sigma_Model

    def fun(x, k, y):
        z = x
        term = np.empty(n)
        for i in range(n):
            term[i] = (Sigma(z, S, k[i], T, Rd, Rf) - y[i])
        return term

    res_lsq = least_squares(fun, 
                            GUESS, 
                            loss = 'cauchy', 
                            bounds=BOUNDS, 
                            args = (INPUT, OBJ), 
                            verbose = Verbose,
                            max_nfev=180
                           )
    x = res_lsq.x
    s = Sigma(x, S, INPUT, T, Rd, Rf)
    
    # 20190816 modify Errors
    o1 = OBJ[0]
    o2 = OBJ[1]
    o3 = OBJ[2]
    o4 = OBJ[3]
    o5 = OBJ[4]

    c1 = s[0]
    c2 = s[1]
    c3 = s[2]
    c4 = s[3]
    c5 = s[4]
    
    # mean squared error
    e1 = ((o1 - c1)  / o1) ** 2
    e2 = ((o2 - c2)  / o2) ** 2
    e3 = ((o3 - c3)  / o3) ** 2
    e4 = ((o4 - c4)  / o4) ** 2
    e5 = ((o5 - c5)  / o5) ** 2

    # 20190822 mean error
    m1 = (np.abs(o1 - c1)  / o1)
    m2 = (np.abs(o2 - c2)  / o2)
    m3 = (np.abs(o3 - c3)  / o3)
    m4 = (np.abs(o4 - c4)  / o4)
    m5 = (np.abs(o5 - c5)  / o5)

    rmse = ((e1+e2+e3+e4+e5) / 5) ** 0.5
    mse = (m1+m2+m3+m4+m5) / 5 

    # 20190816 two errors to one rmse
    # 20190822 rmse, error, errorpa
    error = np.linalg.norm(OBJ-s)
    errorpa = error / OBJ.mean() * 100
    return res_lsq.x, s, rmse, mse, error, errorpa, res_lsq.nfev, res_lsq.status

In [ ]:
def func(row):
    global GUESS
    
    S = row['S']
    Rd = row['Rd']
    Rf = row['Rf']
    T = row['T']
    
    S10P = row['S10P']
    S25P = row['S25P']
    SATM = row['SATM']
    S25C = row['S25C']
    S10C = row['S10C']
    
    K10P = row['K10P']
    K25P = row['K25P']
    KATM = row['KATM']
    K25C = row['K25C']
    K10C = row['K10C']
    
    OBJ = np.array([S10P, S25P, SATM, S25C, S10C])
    INPUT = np.array([K10P, K25P, KATM, K25C, K10C])
    
    res_lsq_x, s,rmse,mse, error, errorpa, res_lsq_nfev, res_lsq_status = Fit_BF_sigmalow(OBJ, INPUT, GUESS, S, T, Rd, Rf)
    print('Data Now is:\n%s\nGUESS = %s\n error = %f\n errorpa = %f' %(row,GUESS, error, errorpa))
    
    GUESS = res_lsq_x

    return pd.Series({
        'sigma0': res_lsq_x[0],
        'sigma1': res_lsq_x[1],
        'alpha': res_lsq_x[2],
        'kappa': res_lsq_x[3],
        
        'sS10P': s[0],
        'sS25P': s[1],
        'sSATM': s[2],
        'sS25C': s[3],
        'sS10C': s[4],
        
        'rmse': rmse,
        'mse': mse,
        'Err': error,
        'ErrPa': errorpa,
        'Nfev': res_lsq_nfev,
        'Status': res_lsq_status
        })

In [ ]:
# 6M
df = pd.read_pickle('./../Data/Data_for_Calibration/USDHKD0608_6M_for_Calibration_Average.pkl')
del df['SEMP'] # delet semp which is ueseless

# 2010~2020 bs2bs usdhkd 6m

In [ ]:
# drop nan value
df = df.dropna()

df.head() # show and check

In [ ]:
GUESS = np.array([0.00255773,  0.13067177,  0.79885264, -0.02901315])

In [ ]:
tic()
result_df = df.apply(func, axis = 1)
toc()

In [ ]:
# 6m 2010~2020: 45009.54560112953 s

In [ ]:
combinedf = df.join(result_df)

In [ ]:
combinedf.columns